In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
%cd /content/drive/My Drive/Colab Notebooks/CNN/샴 안면인식시스템
!pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/CNN/샴 안면인식시스템
/content/drive/My Drive/Colab Notebooks/CNN/샴 안면인식시스템


In [ ]:
#  !git clone https://github.com/PacktPublishing/Neural-Network-Projects-with-Python.git

In [5]:
!python siamese_nn.py

2020-08-13 10:56:46.166192: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-13 10:58:15.250099: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-08-13 10:58:15.299837: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-08-13 10:58:15.299932: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (515feb4febfc): /proc/driver/nvidia/version does not exist
2020-08-13 10:58:15.317388: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2300000000 Hz
2020-08-13 10:58:15.317701: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2418840 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-08-13 10:58:15.317739: I tensorflow/com

# *Haar Cascades** 하르 기반 다단계 검출

밝은 부분과 어두움 부분을 반복하는 패턴을 하르 특징이라고 함


In [6]:
# 오픈 cv는 비전 프로젝트에 사용할 수 있는 파이썬 라이브러리
import cv2
import os

In [ ]:
!pwd

/content/drive/My Drive/Colab Notebooks/CNN/샴 안면인식시스템


In [7]:
face_cascade = cv2.CascadeClassifier( cv2.data.haarcascades +'haarcascade_frontalface_default.xml')
###코드 고침 cv2에 있는 하르 케스케이드 포함시켜줘서 에러해결



def detect_faces(img, draw_box=True):
      # 이미지를 흑백으로 바꾼다.
      grayscale_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

      # 얼굴을 검출한다.
      faces = face_cascade.detectMultiScale(grayscale_img, scaleFactor=1.1,
        minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE)

      face_box, face_coords = None, []

      for (x, y, w, h) in faces:
        if draw_box:
          cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 5)
        face_box = img[y:y+h, x:x+w]
        face_coords = [x,y,w,h]

      return img, face_box, face_coords

if __name__ =="__main__":
      files = os.listdir('sample_faces')
      images = [ file for file in files if 'jpg' in file]
      for image in images:
          img = cv2.imread ( 'sample_faces/' + image)
          detected_faces, _, _ = detect_faces (img)
          cv2.imwrite ('sample_faces/detected_faces/' + image, detected_faces)

Util.py 내용

In [ ]:
'''
얼굴 인식에 필요한 헬퍼 함수
'''
import numpy as np
import random
import os
import cv2
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import load_img, img_to_array

In [ ]:
def euclidean_distance(vectors):
    vector1, vector2 = vectors
    sum_square = K.sum(K.sum_square(vector1 - vector2), axis =1,keepdims= True)
    return K.sqrt(K.maximum( sum_square, K.epsilon()))

def contrastive_loss(Y_true, D):
    margin = 1
    return K.mean( Y_true * K.square(D) + (1 - Y_true) * K.maximum((margin - D),0))

def accuracy( y_true, y_pred):
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))


def create_pairs(X,Y, num_classes):
    pairs, labels = [], []
    # 각 클래스의 이미지를 가리키는 인덱스 배열
    class_idx = [np.where(Y==i)[0] for i in range(num_classes)]
    # 이미지가 가장 적은 클래스의 이미지 개수
    min_images = min(len(class_idx[i]) for i in range(num_classes)) - 1

    
    for c in range(num_classes):
        for n in range(min_images):
            # 양성 쌍을 생성한다
            img1 = X[class_idx[c][n]]
            img2 = X[class_idx[c][n+1]]
            pairs.append((img1, img2))
            labels.append(1)

            # 음성쌍을 생성한다
            # 현재 클래스 c를 제외한 나머지 클래스 리스트
            neg_list = list(range(num_classes))
            neg_list.remove(c)
            # 나머지 클래스 리스트에서 무작위로 하나를 골라 음성쌍에 사용한다
            neg_c = random.sample(neg_list,1)[0]
            img1 = X[class_idx[c][n]]
            img2 = X[class_idx[neg_c][n]]
            pairs.append((img1,img2))
            labels.append(0)

    return np.array(pairs), np.array(labels)

def create_shared_network( input_shape):
    model = Sequential (name= 'Shared_Conv_Network')
    model.add ( Conv2D (filters=64, kernel_size= (3,3),activation='relu',
                        input_shape= input_shape ))
    model.add (MaxPooling2D())
    model.add (Conv2D(filters=64, kernel_size=(3,3),activation='relu'))
    model.add (Flatten())
    model.add (Dense (units= 128,activation= 'sigmoid'))
    return model

def get_data(dir):
    X_train, Y_train = [], []
    X_test, Y_test = [], []
    subfolders = sorted ([file.path for file in os.scandir(dir) if file.is_dir()])
    for idx, folder in enumerator(subfolders):
        for file in sorted( os.listdir(folder)):
            img = load_img( folder+"/"+file, color_mode='grayscale')
            img = img_to_array(img).astype('float32')/255
            img = img.reshape( img.shape[0], img.shape[1], 1 )
            if idx < 35:
                X_train.append(img)
                Y_train.append(idx)
            else:
                X_test.append(img)
                Y_test.append(idx-35)

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    Y_train = np.array(Y_train)
    Y_test = np.array(Y_test)
    return (X_train, Y_train), (X_test, Y_test)

def write_on_frame(frame, text, text_x, text_y):
    (text_width, text_height) = cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, thickness=2)[0]
    box_coords = ((text_x, text_y), (text_x+text_width+20, text_y-text_height-20))
    cv2.rectangle( frame, box_coords[0], box_coords[1], (255,255,255), cv2.FILLED)
    cv2.putText(frame,text, (text_x, text_y-10), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, fontScale=1,color=(0,0,0), thickness=2)
    return frame                          


siamese_nn.py 

In [ ]:
import utils
import numpy as np
from keras.layers import Input,Lambda
from keras.models import Model

In [ ]:
  
'''
안면 인식에 사용할 샴 신경망을 훈련시키는 메인 코드
'''
import utils
import numpy as np
from keras.layers import Input, Lambda
from keras.models import Model

faces_dir = 'att_faces/'

# 훈련 데이터셋과 테스트 데이터셋을 가져온다
(X_train, Y_train), (X_test, Y_test) = utils.get_data(faces_dir)
num_classes = len(np.unique(Y_train))

# 샴 신경망을 만든다
input_shape = X_train.shape[1:]
shared_network = utils.create_shared_network(input_shape)
input_top = Input(shape=input_shape)
input_bottom = Input(shape=input_shape)
output_top = shared_network(input_top)
output_bottom = shared_network(input_bottom)
distance = Lambda(utils.euclidean_distance, output_shape=(1,))([output_top, output_bottom])
model = Model(inputs=[input_top, input_bottom], outputs=distance)

# 모델을 훈련시킨다
training_pairs, training_labels = utils.create_pairs(X_train, Y_train, num_classes=num_classes)
model.compile(loss=utils.contrastive_loss, optimizer='adam', metrics=[utils.accuracy])
model.fit([training_pairs[:, 0], training_pairs[:, 1]], training_labels,
          batch_size=128,
          epochs=10)

# 모델을 저장한다
model.save('siamese_nn.h5')

Epoch 1/10


TypeError: ignored

In [ ]:
'''
안면 인식에 사용할 샴 신경망을 훈련시키는 메인 코드
'''

faces_dir = 'att_faces/'

(X_train, Y_train), (X_test, Y_test) = utils.get_data(faces_dir)
num_classes = len( np.unique(Y_train))

#샴 신경망을 만든다
input_shape = X_train.shape[1:]
shared_network = utils.create_shared_network(input_shape)
input_top = Input(shape=input_shape)
input_bottom = Input(shape=input_shape)
output_top = shared_network(input_top)
output_bottom = shared_network(input_bottom)
distance = Lambda(utils.euclidean_distance,output_shape= (1,))([output_top,output_bottom])
model = Model (inputs= [input_top, input_bottom], outputs = distance)

training_pairs, training_labels = utils.create_pairs (X_train, Y_train, num_classes=num_classes)
model.compile(loss=utils.contrastive_loss, optimizer='adam', metrics=[utils.accuracy])
model.fit([training_pairs[:,0], training_pairs[:,1]], training_labels, batch_size=128, epochs=10)

model.save('siamese_nn.h5')


Epoch 1/10


TypeError: ignored

# **안면인식시스템 전체코드**

face_recognition_system.py

In [ ]:
name = input("What is your name?")

import os
import sys
import cv2
import utils
from keras.models import load_model
import face_detection
import collections

In [8]:
import matplotlib
matplotlib.use("TkAgg")
from keras.preprocessing.image import load_img, img_to_array
import os
import numpy as np
from matplotlib import pyplot as plt

ImportError: ignored